In [3]:
import re
import csv
from math import log, exp
from collections import Counter

# 최종 목표: P(긍정|사용자가 입력한 값), p(부정|사용자가 입력한 값)
# 사용자가 입력한 값 = test_data
# 긍정|부정 데이터셋 = train_data

def start():
    train_datas = open_csv()
#     print(train_datas)
#     test_data = input()
#     prob = naiv_bayes(train_datas,test_datas,0.5,0.5)
    
#     print(f'{test_data}가 부정적일 확률: {prob[0]}, 긍정적일 확률: {prob[1]}')

def open_csv():
    f = open('IMDB_Dataset.csv',encoding='utf-8')
    csvreader = csv.reader(f)
    
    pos_doc = []
    neg_doc = []
    
    # 첫줄 스킵
    next(csvreader)
    for line in csvreader:
        if line[1] == 'positive':
            pos_doc.append(line[0])
        else:
            neg_doc.append(line[0])
    # positive, negative 여부에 따라 각각 리스트에 저장
    train_datas = [[],[]]
    train_datas[0] = neg_doc
    train_datas[1] = pos_doc
    
    return [' '.join(train_datas[0]),' '.join(train_datas[1])]

# P(test_data | 긍정)
# 동전 2개 던져서 앞면 앞면 = 1/4
# 긍정인 데이터셋에서 최고 최고 -> 최고의 빈도수 | 전체 단어의 빈도수 * 최고의 빈도수 | 전체 단어의 빈도수
# 어떤 문장 (test_data) -> 문장이 나올 확률 = 각 단어가 등장할 확률 다 곱한 것
def calculate_doc_prob(train_data, test_data, nowords_weight):
    # 스탑워드 제거
    sw_train_data = re.compile('[^\w]').sub(' ',train_data.lower())
    # 토큰화
    sw_train_token = sw_train_data.split()
    # Bow화 (단어: 빈도수 형태)
    train_vector = dict(Counter(sw_train_token))
    
    # 스탑워드 제거
    sw_test_data = re.compile('[^\w]').sub(' ',test_data.lower())
    # 토큰화
    sw_test_token = sw_test_data.split()
    # Bow화 (단어: 빈도수 형태)
    test_vector = dict(Counter(sw_test_token))
    
    log_prob = 0
    total_wc = len(sw_train_token)
    
    # log(P(test_data|긍정(train_data)))
    # 문장이 나올 확률 = 각 단어가 등장할 확률의 곱
    # 단어가 많아질수록 확률이 매우 작아지므로 로그를 적용한다.
    for word in test_vector:
        if word in train_vector:
            log_prob += log(train_vector[word]/total_wc)
        else:
            log_prob += log(nowords_weight/total_wc)
            # train에 없는 단어에 대한 처리
    return log_prob

def naive_bayes(train_datas, test_data, pos_prob, neg_prob):
    # P(긍정|test_data) = P(test_data|긍정) * P(긍정) / P(test_data)
    test_pos_prob = calculate_doc_prob(train_datas[1],test_data,0.1) + log(pos_prob)
    # P(부정|test_data) = P(test_data|부정) * P(부정) / P(test_data)
    test_neg_prob = calculate_doc_prob(train_datas[0], test_data, 0.1) + log(neg_prob)
    
    # 10:5 -> 2:1
    # 긍정, 부정의 상대적인 크기
    # test_pos_prob(로그값) ->로그값에서 지수값으로 변환해도 0
    # test_neg_prob(로그값) ->로그값에서 지수값으로 변환해도 0
    # 로그함수 ->logex, 지수함수 e^x
    maxprob = max(test_neg_prob, test_pos_prob)
    test_pos_prob -= maxprob
    test_neg_prob -= maxprob
    test_pos_prob = exp(test_pos_prob)
    test_neg_prob = exp(test_neg_prob)
    
    # 두 확률 값의 상대적인 비율
    normalized_prob = [test_neg_prob/(test_pos_prob+test_neg_prob), test_pos_prob/(test_pos_prob+test_neg_prob)]

    return normalized_prob    
    

In [4]:
start()